# Hearthstone Decks: Data Cleaning & Connection to MySQL

The dataset used here is about the Game called "Hearthstone", a card game involves 9 professions & sets of 30 cards of decks.

data.csv includes information about all the complete(30 cards set of cards)decks within a period. data2.csv includes detailed information about all the cards in each sets within the same period.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import matplotlib.pyplot as plt
import datetime

In [2]:
from datetime import datetime, timedelta
from collections import OrderedDict

In [3]:
#open data file which contains decks information
decks = pd.read_csv('data1.csv',encoding='latin-1')

In [4]:
#open data2 file which contains
decks_ref=pd.read_csv('data2.csv', low_memory=False)

In [5]:
#####test
cards_ref=pd.read_json('refs.json')

In [6]:
# reformat data: date string to datetime object
decks['date']=pd.to_datetime(decks['date'])

In [7]:
# card information reference document,including all the cards information in Hearthstone
with open ('refs.json',encoding='utf-8') as file:
    cards=json.load(file)

In [8]:
cards_ref=pd.DataFrame(cards)

## Data Cleaning

In [9]:
#show row of decks before cleaning
raw_len=len(decks)
print(raw_len)

62996


In [10]:
#Check data validity 1 Since the game has officially started from Mar 15,2014, remove decks before the date.
release_date=datetime(2014,3,15)
decks=decks[decks['date']>release_date]

print(len(decks))
print ('First deck :', min(decks['date']))
print ('Last deck :', max(decks['date'])) 

62996
First deck : 2016-01-28 00:00:00
Last deck : 2017-01-19 00:00:00


In [11]:
#Check data validity 2. Duplicity check.
assert len(decks['deck_id'].unique())==len(decks)

In [12]:
#Check data accuracy，check null.
decks.isnull().sum()

craft_cost        0
date              0
deck_archetype    0
deck_class        0
deck_format       0
deck_id           0
deck_set          0
deck_type         0
rating            0
title             1
user              0
dtype: int64

In [13]:
#delete the rows contains null
decks=decks.dropna()
decks.isnull().sum()

craft_cost        0
date              0
deck_archetype    0
deck_class        0
deck_format       0
deck_id           0
deck_set          0
deck_type         0
rating            0
title             0
user              0
dtype: int64

In [14]:
#Check data accuracy by check duplicity 
assert len(decks_ref['deck_id'].unique())==len(decks_ref)

In [15]:
#Check data completeness.Check whether all 9 deck_class(player's profession) included.
decks['deck_class'].value_counts()

Mage       7869
Priest     7843
Warrior    7438
Paladin    7408
Rogue      6837
Druid      6615
Shaman     6529
Warlock    6472
Hunter     5984
Name: deck_class, dtype: int64

In [16]:
#Check data uniformity.Check whether all the released (featured expansion) included.
decks['deck_set'].value_counts()

Old Gods         24386
Explorers        18386
Karazhan          6933
Yogg Nerf         6476
Classic Nerfs     3471
Gadgetzan         3343
Name: deck_set, dtype: int64

Decks are generally balanced featured in different expansions,so let's continue.

In [17]:
#Check data uniformity.
decks['deck_type'].value_counts()

Ranked Deck      38132
None             15794
Theorycraft       4153
Tavern Brawl      1731
Arena             1366
PvE Adventure     1235
Tournament         584
Name: deck_type, dtype: int64

There are different mode of gaming in Hearthstone.
PvE,Tavern Brawl and Arena invole decks that are randomly built by system not the player, so we will get rid of them.Theorycraft means praticing sets with unreleased expansion so I get rid of that too.

In this sense, only Ranked Deck and Tournament are the decks built to PVP, so we will only analyze these two.

In [18]:
Ranked_decks=decks[decks['deck_type']=='Ranked Deck']
Tournament_decks=decks[decks['deck_type']=='Tournament']
print (len(Ranked_decks))
print (len(Tournament_decks))

38132
584


In [19]:
import MySQLdb#Concat 2 decks collection
decks=pd.concat([Ranked_decks,Tournament_decks])
len(decks)

38716

## Connect to MySQL

### Insert table in MySQL

In [20]:
 from sqlalchemy import create_engine
conn = create_engine('mysql+mysqldb://root:your password@localhost:3306/your schema?charset=utf8')

In [21]:
#create table"decks" in MySQL 
pd.io.sql.to_sql(decks,'decks',con=conn,if_exists='replace')

C:\Users\HP\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py:470: Warning: (1366, "Incorrect string value: '\\xB6\\xAB\\xB2\\xBF\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 481")
  cursor.execute(statement, parameters)
C:\Users\HP\Anaconda3\lib\site-packages\sqlalchemy\dialects\mysql\base.py:1569: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  cursor.execute('SELECT @@tx_isolation')


In [23]:
#create table"decks_ref" in MySQL
pd.io.sql.to_sql(decks_ref,'decks_ref',con=conn,if_exists='replace')

### User Case

In [24]:
#connect to MySQL
import MySQLdb

db = MySQLdb.connect("localhost","root","your password","assignment1" )

In [25]:
# Get cursor
cursor = db.cursor()

In [27]:
# Set Primary Key
cursor.execute("alter table decks add constraint PK1 primary key(deck_id)")

0

In [28]:
cursor.execute("alter table decks_ref add constraint PK2 primary key(deck_id)")

0

### User case 1:Find the top 5 most used "Beginning Card" across all Mage decks

In [29]:
#get cursor
cursor=db.cursor()

In [34]:
#return 5 most used beginning card in 
cursor.execute(" select card_0 as beginning_card, count(*) as times from decks_ref a inner join (select deck_id from decks where deck_class='Mage') b on a.deck_id=b.deck_id group by card_0 order by times desc limit 5")

5

In [35]:
#the output format will be('card_id_1':count);(('card_id_2':count))....
result=cursor.fetchall()
print('(card_id :count)',result)

(card_id :count) (('77', 2166), ('315', 745), ('138', 638), ('113', 407), ('195', 189))


### User case 2: Show the most used "archetyped decks" & it's card setting

In [36]:
# return the 5 most used featured decks(deck_archetype is not null)
cursor.execute("select deck_archetype,deck_class,count(*) from decks where deck_archetype !='Unknown'group by deck_archetype order by count(*) desc limit 5")

5

In [37]:
#Format:('deck_archetype','class','count')
result2=cursor.fetchall()
print('(deck_archetype,class,count)',result2)

(deck_archetype,class,count) (('Control Warrior', 'Warrior', 1586), ('Midrange Shaman', 'Shaman', 1552), ('Control Priest', 'Priest', 1426), ('Midrange Hunter', 'Hunter', 1274), ('Tempo Mage', 'Mage', 1260))


In [38]:
#show most used ultimate card in 'Midrange Hunter' decks;format:
cursor.execute("select card_29, count(*) as times from decks_ref a inner join(select deck_id from decks where deck_archetype='Midrange Hunter') b on a.deck_id=b.deck_id group by card_29 order by times desc limit 5")
result2_2=cursor.fetchall()
print('(Card_Id, Count) ', result2_2)

(Card_Id, Count)  ((38985.0, 345), (39481.0, 290), (38734.0, 260), (41126.0, 117), (38910.0, 51))


## Usercase 3: show the detail information of the first 10  beginning cards in hunter decks


In [39]:
# worked in MySQL and due to the db module problem can't be done here;
#show detail card_0 information and number in a hunter decks

cursor.execute("select Card_Name, Rarity, Card_Class from card_refs c join decks_ref dr on c.Card_Id=dr.card_0 where playerClass='HUNTER' LIMIT 10;")
result3=cursor.fetchall()
print(result3)


(('Tundra Rhino', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'), ('Houndmaster', 'FREE', 'HUNTER'), ('Houndmaster', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'), ('Tundra Rhino', 'FREE', 'HUNTER'))


### Usercase 4: Count the type of beginning card（Card_0）

In [40]:
# return the type of card_0 and count
cursor.execute("select Card_Type,count(*) from card_refs c join decks_ref d on c.Card_Id=d.card_0 group by Card_Type;")
result4=cursor.fetchall()
print (result4)

(('MINION', 19202),)


### Usercase 5: the count of the beginning card types

In [41]:
cursor.execute("select Card_Class, count(*) as times from card_refs c join decks_ref d on c.Card_Id=d.card_0 group by Card_Class order by times desc")
result5=cursor.fetchall()
print (result5)

(('NEUTRAL', 11372), ('DRUID', 7250), ('SHAMAN', 552), ('HUNTER', 24), ('ROGUE', 1), ('WARRIOR', 1), ('PRIEST', 1), ('MAGE', 1))
